In [1]:
import pandas as pd

In [2]:
arquivoRegiaoIBGE = '../Dengue/Regiao/regiao.xlsx'

In [59]:
dfRegiao = pd.read_excel(arquivoRegiaoIBGE, sheet_name='IBGE', usecols=[11,12], names=['geocode','nome_municipio'])

In [60]:
dfRegiao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   geocode         92 non-null     int64 
 1   nome_municipio  92 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.6+ KB


In [6]:
dfRegiao

,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município,Regiao
0,33,Rio de Janeiro,3301,Rio de Janeiro,330002,Angra dos Reis,5,Sul Fluminense,13,Baía da Ilha Grande,100,3300100,Angra dos Reis,B. DA ILHA GRANDE
1,33,Rio de Janeiro,3304,Campos dos Goytacazes,330012,Santo Antônio de Pádua,1,Noroeste Fluminense,2,Santo Antônio de Pádua,159,3300159,Aperibé,NOROESTE
2,33,Rio de Janeiro,3305,Macaé - Rio das Ostras - Cabo Frio,330013,Cabo Frio,4,Baixadas,10,Lagos,209,3300209,Araruama,BAIXADA LITORÂNEA
3,33,Rio de Janeiro,3303,Petrópolis,330007,Petrópolis,3,Centro Fluminense,5,Três Rios,225,3300225,Areal,CENTRO-SUL
4,33,Rio de Janeiro,3305,Macaé - Rio das Ostras - Cabo Frio,330013,Cabo Frio,4,Baixadas,10,Lagos,233,3300233,Armação dos Búzios,BAIXADA LITORÂNEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,33,Rio de Janeiro,3303,Petrópolis,330009,Três Rios - Paraíba do Sul,3,Centro Fluminense,5,Três Rios,6008,3306008,Três Rios,CENTRO-SUL
88,33,Rio de Janeiro,3302,Volta Redonda - Barra Mansa,330006,Valença,5,Sul Fluminense,12,Barra do Piraí,6107,3306107,Valença,MÉDIO PARAÍBA
89,33,Rio de Janeiro,3304,Campos dos Goytacazes,330011,Itaperuna,1,Noroeste Fluminense,1,Itaperuna,6156,3306156,Varre-Sai,NOROESTE
90,33,Rio de Janeiro,3302,Volta Redonda - Barra Mansa,330006,Valença,6,Metropolitana do Rio de Janeiro,14,Vassouras,6206,3306206,Vassouras,CENTRO-SUL


In [23]:

ew_start = '2022'
ew_end = '2023'


dfAPI = pd.read_csv('https://info.dengue.mat.br/api/alertcity?geocode=3304557&disease=dengue&format=csv&ew_start=1&ew_end=53&ey_start=2022&ey_end=2023')





In [65]:

ey_start = '2022'
ey_end = '2023'

for geocode,nome_municipio   in dfRegiao.itertuples(index=False):
    
    nome_municipio_t = nome_municipio.replace(' ','_')
    
    dfAPI = pd.read_csv(fr'https://info.dengue.mat.br/api/alertcity?geocode={geocode}&disease=dengue&format=csv&ew_start=1&ew_end=53&ey_start={ey_start}&ey_end={ey_end}')
   
    dfAPI['Nome_Município'] = nome_municipio
    dfAPI['codigo_municipio_completo'] = geocode
           
    arquivo = fr'../Dengue/Arquivos/arquivo_dengue_{geocode}_{nome_municipio_t}.csv'
    
    dfAPI.to_csv(arquivo, index=False, sep=';')

3300100
Angra dos Reis
3300159
Aperibé
3300209
Araruama
3300225
Areal
3300233
Armação dos Búzios
3300258
Arraial do Cabo
3300308
Barra do Piraí
3300407
Barra Mansa
3300456
Belford Roxo
3300506
Bom Jardim
3300605
Bom Jesus do Itabapoana
3300704
Cabo Frio
3300803
Cachoeiras de Macacu
3300902
Cambuci
3301009
Campos dos Goytacazes
3301108
Cantagalo
3300936
Carapebus
3301157
Cardoso Moreira
3301207
Carmo
3301306
Casimiro de Abreu
3300951
Comendador Levy Gasparian
3301405
Conceição de Macabu
3301504
Cordeiro
3301603
Duas Barras
3301702
Duque de Caxias
3301801
Engenheiro Paulo de Frontin
3301850
Guapimirim
3301876
Iguaba Grande
3301900
Itaboraí
3302007
Itaguaí
3302056
Italva
3302106
Itaocara
3302205
Itaperuna
3302254
Itatiaia
3302270
Japeri
3302304
Laje do Muriaé
3302403
Macaé
3302452
Macuco
3302502
Magé
3302601
Mangaratiba
3302700
Maricá
3302809
Mendes
3302858
Mesquita
3302908
Miguel Pereira
3303005
Miracema
3303104
Natividade
3303203
Nilópolis
3303302
Niterói
3303401
Nova Friburgo
3303500
N